# CNN

In [1]:
import torch

In [2]:
torch.cuda.device(0)

In [3]:
torch.cuda.get_device_name()

'Tesla T4'

In [4]:
torch.cuda.is_available() #check GPU是否可用

True

In [5]:
num_epochs = 5
batch_size = 100
learning_rate = 0.001

In [6]:
import torchvision.datasets as dsets
import torchvision.transforms as transforms #把資料轉成tenser方便計算

In [7]:
train_dataset = dsets.MNIST(root='./data',  #放在home下面data資料夾
                  train=True,
                  transform=transforms.ToTensor(),  #轉成tenser
                  download=True) #存在目錄下->重新load會比較快

test_dataset = dsets.MNIST(root='./data',  #放在home下面data資料夾
                  train=False, #不用train不用train
                  transform=transforms.ToTensor(),  #轉成tenser
                  ) #預設不download

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw



In [20]:
train_dataset

Dataset MNIST
    Number of datapoints: 60000
    Root location: ./data
    Split: Train
    StandardTransform
Transform: ToTensor()

In [8]:
import torch.utils.data as Data

In [9]:
train_loader = Data.DataLoader(dataset = train_dataset,
                 batch_size = batch_size,
                 shuffle = True)  #shuffle data才不會讀到重覆的batch->訓練效果較好
test_loader = Data.DataLoader(dataset= test_dataset,
                 batch_size = batch_size,
                 shuffle=False)  #不做shuffle data

In [10]:
import torch.nn as nn

In [11]:
class Model(nn.Module):
  def __init__(self):
    super(Model, self).__init__()
    self.conv = nn.Sequential(
        nn.Conv2d(1,64,kernel_size = 3, stride = 1, padding = 1), #處理2d圖片 (input_channel, output_channel, filter_size, 步伐大小,padding)
        nn.ReLU(),
        #先不做max_pooling
        nn.Conv2d(64, 128, kernel_size=3, padding = 1), #將第一層掃過的低階特徵組合成高階特徵
        nn.ReLU(),
        nn.MaxPool2d(stride = 2, kernel_size =2) #檢查的區塊不重複(否則較強特徵會被放大)->padding = 區塊大小
    )
    self.dense = nn.Sequential(
        nn.Linear(14*14*128, 1024),
        nn.ReLU(), #隱藏層
        nn.Linear(1024, 10)
    )
  def forward(self, x):
    out = self.conv(x)
    out = out.view(-1,14*14*128) #攤平
    out = self.dense(out)
    return out


In [12]:
model = Model()
model.cuda() #把模型送入GPU把模型送入GPU

Model(
  (conv): Sequential(
    (0): Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU()
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (dense): Sequential(
    (0): Linear(in_features=25088, out_features=1024, bias=True)
    (1): ReLU()
    (2): Linear(in_features=1024, out_features=10, bias=True)
  )
)

In [13]:
import torch.optim as opt

In [14]:
loss_fn = nn.CrossEntropyLoss() #分類用CrossEntropy
optimizer = opt.Adam(model.parameters(), lr = learning_rate)

In [15]:
from torch.autograd import Variable

In [16]:
for epoch in range(num_epochs):
  for i, (images, labels) in enumerate(train_loader):
   images = Variable(images.type(torch.cuda.FloatTensor)) #資料轉型
   labels = Variable(labels.type(torch.cuda.LongTensor))
   optimizer.zero_grad()
   outputs = model(images)
   loss = loss_fn(outputs, labels) #預測結果,答案
   loss.backward()
   optimizer.step()
   if (i+1)%100 == 0:
     print("Epoch:%d, Batch:%d, Loss:%.4f" 
        %(epoch+1, i+1, loss.data))

Epoch:1, Batch:100, Loss:0.1699
Epoch:1, Batch:200, Loss:0.0447
Epoch:1, Batch:300, Loss:0.0573
Epoch:1, Batch:400, Loss:0.1581
Epoch:1, Batch:500, Loss:0.0173
Epoch:1, Batch:600, Loss:0.0495
Epoch:2, Batch:100, Loss:0.0148
Epoch:2, Batch:200, Loss:0.0078
Epoch:2, Batch:300, Loss:0.0845
Epoch:2, Batch:400, Loss:0.0363
Epoch:2, Batch:500, Loss:0.0742
Epoch:2, Batch:600, Loss:0.0270
Epoch:3, Batch:100, Loss:0.0028
Epoch:3, Batch:200, Loss:0.0281
Epoch:3, Batch:300, Loss:0.0040
Epoch:3, Batch:400, Loss:0.0342
Epoch:3, Batch:500, Loss:0.0131
Epoch:3, Batch:600, Loss:0.0303
Epoch:4, Batch:100, Loss:0.0015
Epoch:4, Batch:200, Loss:0.0005
Epoch:4, Batch:300, Loss:0.0026
Epoch:4, Batch:400, Loss:0.0504
Epoch:4, Batch:500, Loss:0.0055
Epoch:4, Batch:600, Loss:0.0056
Epoch:5, Batch:100, Loss:0.0053
Epoch:5, Batch:200, Loss:0.0071
Epoch:5, Batch:300, Loss:0.0058
Epoch:5, Batch:400, Loss:0.0180
Epoch:5, Batch:500, Loss:0.0006
Epoch:5, Batch:600, Loss:0.0001


In [17]:
import torch

In [18]:
correct = 0
total = 0
for images, labels in test_loader:
  images = Variable(images.type(torch.cuda.FloatTensor))
  outputs = model(images)
  _, pred = torch.max(outputs.data, 1) #_:最大值,pred:位置
  correct += (pred == labels.type(torch.cuda.LongTensor)).sum()
  total += labels.size(0)
print("Accuracy:%.3f%%" %(100.0* float(correct)/float(total)))

Accuracy:99.040%
